In [3]:
import pandas as pd
import numpy as np
import pickle
import datetime
import json
import re
import os

#### Files used
- SurveyQuestionResults
    - survey_daily-check-in-detail
    - survey_daily-check-in-general
    - survey_daily-affect-custom-evening
    - survey_daily-affect-custom-morning


### Variable Information

In [4]:
## Concerned with survey_question_results

In [5]:
## Input Directory
directory = "/Users/farhan/DNL/BuddingScholar/Budding_Scholar_22-23/Data"

In [6]:
## Output Directory
out_dir = "../indv_table_exports/"

In [7]:
## Good Participants
good_subjects = ['1e7aef96-16cc-43f8-95d4-e3bc582eb6d3', '2baee05a-5e5a-4436-8c25-2628d46d1e08']

### Participant List

In [8]:
df_participants = pd.read_csv("/Users/farhan/DNL/BuddingScholar/Budding_Scholar_22-23/Data/20221025-20221026/StudyParticipants_20221026.csv")
df_participants["CustomFields"] = df_participants["CustomFields"].apply(json.loads)

participant_list = []
for index, row in df_participants.iterrows():
    if row["CustomFields"]["exp_version"] == "run_1":
        participant_list.append(row["ParticipantIdentifier"])

### Some handy functions

In [9]:
## Function to fix date from UTC to ET
def fix_date_to_ET(end_date):
    if pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').hour < 5:
        return pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date() - datetime.timedelta(days=1)
    else:
        return pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date()
            
def fix_columns_by_category(dataframe, categories):
    df = dataframe
    for item in categories:
        if item not in df.columns.to_list():
            df[item] = 'NaN'
    return df

def pivot_df(dataframe, pos: list, col, val):
    df = dataframe
    df = df.pivot_table(index=pos,
                    columns=col, 
                    values=val).reset_index()
    return df

def get_time_diff(d2, d1):
    return (d2 - d1).days

def get_unique_participant_count(dataframe):
    temp = []
    for item in dataframe["ParticipantIdentifier"].to_list():
        if item not in temp:
            temp.append(item)
    return (len(temp))

def get_absent_participants(dataframe, participant_list):
    ## Absent participants
    absent_participants = []

    ## Get participants not found in df_samples
    temp = []
    for item in dataframe["ParticipantIdentifier"].to_list():
        if item not in temp:
            temp.append(item)

    for item in participant_list:
        if item not in temp:
            absent_participants.append(item)
    
    return absent_participants

### Dataframe Construction Functions

In [10]:
## Function to extract Affect scores
def get_affect_df(dataframe):
    df = dataframe
    df_affect = df.loc[df.ResultIdentifier.str.startswith('affect_'), :]
    df_affect = df_affect[['ParticipantIdentifier', 'ResultIdentifier', 'Answers', 'StudyDay']]
    
    # cast Value to numeric
    df_affect.Answers = pd.to_numeric(df_affect.Answers)

    # Make separate columns for each affect score
    # Convert from long to wide
    indices = ['StudyDay', 'ParticipantIdentifier']
    df_affect = pivot_df(df_affect, indices, 'ResultIdentifier', 'Answers')
    
    # Add columns that may not have been present in given dataset
    evening_indices = ['affect_neg_angry', 'affect_neg_ashamed', 'affect_neg_bored', 'affect_neg_depressed', 
               'affect_neg_embarrassed', 'affect_neg_frustrated', 'affect_neg_guilty', 'affect_neg_lazy',
               'affect_neg_lonelyIsolated', 'affect_neg_nervousAnxious', 'affect_neg_sad', 'affect_neg_stressed',
               'affect_pos_amused', 'affect_pos_appreciated', 'affect_pos_excited', 'affect_pos_focused', 
               'affect_pos_happy', 'affect_pos_hopeful', 'affect_pos_motivated', 'affect_pos_relaxedCalm']

    morning_indices = ["affect_neg_frustrated_am", "affect_pos_relaxedCalm_am", "affect_neg_ashamed_am", 
                "affect_pos_excited_am", "affect_neg_stressed_am", "affect_pos_motivated_am", "affect_neg_angry_am",
                "affect_pos_happy_am", "affect_neg_nervousAnxious_am", "affect_pos_focused_am", "affect_neg_lonelyIsolated_am",
                "affect_neg_guilty_am", "affect_pos_appreciated_am", "affect_neg_depressed_am", "affect_pos_hopeful_am",
                "affect_neg_sad_am", "affect_pos_amused_am", "affect_neg_bored_am", "affect_neg_embarrassed_am", "affect_neg_lazy_am"]

    df_affect = fix_columns_by_category(df_affect, evening_indices)
    df_affect = fix_columns_by_category(df_affect, morning_indices)
    
    # Rename columns (add prefix SR for self report)
    keep_same = {'StudyDay', 'ParticipantIdentifier'}
    df_affect.columns = ['SR_' + str(col) if col not in keep_same else col for col in df_affect.columns]

    return df_affect

## Function to get goal, past24, next24 dataframe
def get_goals_df(survey_dataframe):
    df_survey = survey_dataframe
    df = df_survey.loc[df_survey.ResultIdentifier.str.startswith('DAILY_'), :]
    df = df[['ParticipantIdentifier', 'ResultIdentifier', 'Answers', 'StudyDay']].reset_index()

    ## Drop duplicate rows # This is hacky ## Show this to daniel ## Date conversion problem
    df = df.drop_duplicates(subset=['ParticipantIdentifier', 'ResultIdentifier', 'StudyDay'], keep="first")
    
    ## Make columns for each measure found in the ResultIdentifier column
    indices = ['StudyDay', 'ParticipantIdentifier']
    df = df.pivot_table(index=indices,
                        columns='ResultIdentifier', 
                        values='Answers',
                        aggfunc=lambda x: ' '.join(x)).reset_index()
    
    ## Dataframe containing goals
    df_goals = df
    
    ## Extract dfs of only past24 and next24 data
    past_24_col = [col for col in df_goals if (col.startswith('DAILY_past24') or col in indices)]
    next_24_col = [col for col in df_goals if (col.startswith('DAILY_next24') or col in indices)]

    df_past24 = df[past_24_col]
    df_next24 = df[next_24_col]
    
    return df_goals, df_past24, df_next24

## Function to calculate gap for one participant
def get_participant_gap_df(df_next24i, df_past24i):
    ## This will store the final gap df
    df_gap = []
    
    ## List of some important column names
    ## Will prove useful during iterations
    gap_cols = ['drinks', 'exercise', 'leisureNonSolo', 'leisureSolo', 'nonoccupation', 
                'occupation', 'sleep', 'socialMedia']
    next_cols = ['DAILY_next24_drinks', 'DAILY_next24_exercise', 'DAILY_next24_leisureNonSolo', 'DAILY_next24_leisureSolo',
                       'DAILY_next24_nonoccupation', 'DAILY_next24_occupation', 'DAILY_next24_sleep', 'DAILY_next24_socialMedia']
    past_cols = ['DAILY_past24_drinks', 'DAILY_past24_exercise', 'DAILY_past24_leisureNonSolo', 'DAILY_past24_leisureSolo',
                       'DAILY_past24_nonoccupation', 'DAILY_past24_occupation', 'DAILY_past24_sleep', 'DAILY_past24_socialMedia']                   

    
    ## Input dataframes
    df_next24 = df_next24i
    df_past24 = df_past24i
    
    ## Loop over each row in df_next24
    for i in range(len(df_next24)):
        
        ## This will hold gap results for one row
        row_gap_dict = {}

        ## Append participant ID and StudyDay
        row_gap_dict['StudyDay'] = df_next24.at[i, 'StudyDay']
        row_gap_dict['ParticipantIdentifier'] = df_next24.at[i, 'ParticipantIdentifier']

        for item in gap_cols:

            if i == 0:
                ## There is no previous day for gap to be calculated
                row_gap_dict[item + '_gap'] = 'NaN'
                
            elif df_next24.at[i, next_cols[gap_cols.index(item)] + '_goal'] == 'False':
                ## This was not set as a goal
                row_gap_dict[item + '_gap'] = 'NaN'
                
            else:

                ## get the date of the previous row in past24 dataframe
                past_row_date = df_past24.at[i - 1, 'StudyDay']

                ## Get current row date of the next24 dataframe
                current_date = df_next24.at[i, 'StudyDay']

                ## Calculate time difference in days
                delta = get_time_diff(current_date, past_row_date)

                ## If the previous day does not exist, gap is NaN
                if delta != 1:
                    row_gap_dict[item + '_gap'] = 'NaN'
                else:
                    ## Calculate gap as required information exists
                    today = float(df_next24.at[i, next_cols[gap_cols.index(item)]])
                    yesterday = float(df_past24.at[i - 1, past_cols[gap_cols.index(item)]])
                    gap = yesterday - today
                    row_gap_dict[item + '_gap'] = gap

        df_gap.append(row_gap_dict)
    df_gap = pd.DataFrame(df_gap)
    return df_gap

### Got all SurveyQuestionResults till date in a dataframe

In [11]:
## Got all SurveyQuestionResults till date in a dataframe
df_survey_list = []
for folder in os.listdir(directory):
    survey_file_name = ""
    path = directory + "/" + folder
    for f_name in os.listdir(path):
        if f_name.startswith("SurveyQuestionResults"):
            survey_file_name = f_name
            break
    path = path + '/' + survey_file_name
    
    current_df = pd.read_csv(path) 
    current_df["StudyDay"] = 0
    current_df["StudyDay"] = current_df.apply(lambda x: fix_date_to_ET(x.EndDate), axis=1)
    df_survey_list.append(current_df)
    
df_survey = pd.concat(df_survey_list)

In [12]:
df_survey

,SurveyQuestionResultKey,SurveyStepResultKey,SurveyResultKey,ParticipantIdentifier,ResultIdentifier,Answers,StartDate,EndDate,StudyDay
0,174b976a-b54b-ed11-aac0-0afb9334277d,fa4a976a-b54b-ed11-aac0-0afb9334277d,f14a976a-b54b-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,WEEKLY_goal_report1,98,2022-10-14T07:41:45-04:00,2022-10-14T07:41:49-04:00,2022-10-14
1,1a4b976a-b54b-ed11-aac0-0afb9334277d,fb4a976a-b54b-ed11-aac0-0afb9334277d,f14a976a-b54b-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,WEEKLY_goal_report2,95,2022-10-14T07:41:49-04:00,2022-10-14T07:41:51-04:00,2022-10-14
2,1f4b976a-b54b-ed11-aac0-0afb9334277d,fd4a976a-b54b-ed11-aac0-0afb9334277d,f14a976a-b54b-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,WEEKLY_goal_set1,Study for midterm,2022-10-14T07:41:52-04:00,2022-10-14T07:41:58-04:00,2022-10-14
3,234b976a-b54b-ed11-aac0-0afb9334277d,fe4a976a-b54b-ed11-aac0-0afb9334277d,f14a976a-b54b-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,WEEKLY_goal_set1_importance,7,2022-10-14T07:41:58-04:00,2022-10-14T07:42:00-04:00,2022-10-14
4,274b976a-b54b-ed11-aac0-0afb9334277d,ff4a976a-b54b-ed11-aac0-0afb9334277d,f14a976a-b54b-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,WEEKLY_goal_set1_consequences,7,2022-10-14T07:42:00-04:00,2022-10-14T07:42:02-04:00,2022-10-14
...,...,...,...,...,...,...,...,...,...
10275,e12782d7-9b4f-ed11-aac0-0afb9334277d,932782d7-9b4f-ed11-aac0-0afb9334277d,6d2782d7-9b4f-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,affect_neg_sad_am,1,2022-10-19T06:50:23-04:00,2022-10-19T06:50:24-04:00,2022-10-19
10276,e42782d7-9b4f-ed11-aac0-0afb9334277d,952782d7-9b4f-ed11-aac0-0afb9334277d,6d2782d7-9b4f-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,affect_pos_amused_am,1,2022-10-19T06:50:24-04:00,2022-10-19T06:50:26-04:00,2022-10-19
10277,e92782d7-9b4f-ed11-aac0-0afb9334277d,962782d7-9b4f-ed11-aac0-0afb9334277d,6d2782d7-9b4f-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,affect_neg_bored_am,1,2022-10-19T06:50:26-04:00,2022-10-19T06:50:28-04:00,2022-10-19
10278,ec2782d7-9b4f-ed11-aac0-0afb9334277d,972782d7-9b4f-ed11-aac0-0afb9334277d,6d2782d7-9b4f-ed11-aac0-0afb9334277d,151eec8c-0ba1-48ba-afc8-f2874dad3a2b,affect_neg_embarrassed_am,1,2022-10-19T06:50:28-04:00,2022-10-19T06:50:29-04:00,2022-10-19


In [13]:
## df_survey = current_df.loc[current_df["ParticipantIdentifier"].isin(good_subjects)]

### Construct the Affect and Goals dataframes

In [14]:
## Got Affect Dataframe
df_affect = get_affect_df(df_survey)

## Get goals, past24, and next_24 Dataframes
df_goals, df_past24, df_next24 = get_goals_df(df_survey)

### Construct the gap Dataframe

In [15]:
## Will get this by concatenating gap dataframes from each good participant
df_gap_list = []
good_subjects = participant_list
for item in good_subjects:
    df_past24_temp = df_past24[df_past24.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    df_next24_temp = df_next24[df_next24.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    current_participant_df_gap = get_participant_gap_df(df_next24_temp, df_past24_temp)
    
    df_gap_list.append(current_participant_df_gap)
df_gap = pd.concat(df_gap_list)

### Combine the three dataframes to make SR Dataframe and Export CSV

In [16]:
## Merge df_goals, df_affect, and df_gap Dataframes
df_self_report = df_affect
df_self_report = df_self_report.merge(df_gap, how='left', on=['ParticipantIdentifier', 'StudyDay'])
print(get_unique_participant_count(df_self_report))

86


In [17]:
## Export the self-report Dataframe as a CSV
df_self_report
df_self_report.to_csv('self_report.csv', index=False)

In [18]:
pd.set_option('display.max_rows', None)
#df_self_report = df_self_report.groupby(["ParticipantIdentifier"], as_index=true).reset_index()
df_self_report.sort_values(by=["ParticipantIdentifier", "StudyDay"])

,StudyDay,ParticipantIdentifier,SR_affect_neg_angry,SR_affect_neg_angry_am,SR_affect_neg_ashamed,SR_affect_neg_ashamed_am,SR_affect_neg_bored,SR_affect_neg_bored_am,SR_affect_neg_depressed,SR_affect_neg_depressed_am,...,SR_affect_pos_relaxedCalm,SR_affect_pos_relaxedCalm_am,drinks_gap,exercise_gap,leisureNonSolo_gap,leisureSolo_gap,nonoccupation_gap,occupation_gap,sleep_gap,socialMedia_gap
0,2022-10-06,01801252-3a7e-4f5f-8b6d-49e8da3902f3,2.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,2022-10-10,01801252-3a7e-4f5f-8b6d-49e8da3902f3,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,2022-10-11,01801252-3a7e-4f5f-8b6d-49e8da3902f3,2.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,2022-10-12,01801252-3a7e-4f5f-8b6d-49e8da3902f3,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293,2022-10-13,01801252-3a7e-4f5f-8b6d-49e8da3902f3,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,2022-10-15,01801252-3a7e-4f5f-8b6d-49e8da3902f3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,...,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507,2022-10-16,01801252-3a7e-4f5f-8b6d-49e8da3902f3,1.0,NaN,2.0,NaN,1.0,NaN,1.0,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,2022-10-17,01801252-3a7e-4f5f-8b6d-49e8da3902f3,2.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718,2022-10-19,01801252-3a7e-4f5f-8b6d-49e8da3902f3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
789,2022-10-20,01801252-3a7e-4f5f-8b6d-49e8da3902f3,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
